In [191]:
import numpy as np
import math
import gurobipy as gp
from gurobipy import GRB

### Preprocessing

TODO:
* Define $u_i$, utility functions
* Define $c$, course capacities
* Define $b_0$, initial budgets    
* Define $K$, max. bundle size

In [192]:
# K is max. number of classes
K = 2
c = [1, 1, 1, 1]
b = [1, 1]
u = [[10, 5, 4, 3]]

In [193]:
# combinations of length n
def combinations(array, tuple_length, prev_array=[]):
    if len(prev_array) == tuple_length:
        return [prev_array]
    combs = []
    for i, val in enumerate(array):
        prev_array_extended = prev_array.copy()
        prev_array_extended.append(val)
        combs += combinations(array[i+1:], tuple_length, prev_array_extended)
    return combs

# all bundles below a maximum length. Greedy version, definitely a better way exists
def powerset(omega, max_length):
    subsets = []
    for i in range(max_length):
        subsets += combinations(omega, i+1)
    return subsets

# turns a bundle into corresponding allocation vector in R^m
def bundle_to_allocation(bundle):
    allocation = np.zeros(len(c))
    for i in bundle:
        allocation[i-1] = 1
    return allocation

# costs for an allocation vector
def cost(allocation, prices):
    return np.dot(allocation, prices)

In [194]:
# Input: Max bundle size
# Output: set of possible allocation vectors
def allocations(max_length):
    omega = np.array(range(1, len(c)+1))
    feasible_bundles = powerset(omega, max_length)
    allocations = []
    for i in feasible_bundles:
        allocations.append(bundle_to_allocation(i))
    return allocations

In [221]:
# input: set of allocations, utility, prices, and budget
# output: utility-maximizing budget-feasible bundles

# note: not always unique
def demand_set(allocations, prices, utility, budget):
    demand_set = []

    max_u = 0
    for i in allocations:
        feasible = (cost(i, prices) <= budget)
        if feasible:
            u = np.dot(i, utility)
            if u < max_u:
                continue
            elif u == max_u:
                demand_set.append(i)
            elif u > max_u:
                max_u = u
                demand_set.clear()
                demand_set.append(i)
    
    print(demand_set)
    return np.asarray(demand_set)

In [219]:
#Please make e/d a whole number for now

def budget_intervals(bundles, prices, utility, budget, delta, epsilon):
    discontinuities = []
    allocations = []
    lower = budget-epsilon
    discontinuities.append(lower)
    current_demand = demand_set(bundles, prices, utility, lower)

    #find closest multiple of delta to lower
    current_budget = lower

    while (current_budget < budget+epsilon):
        new_budget = current_budget + delta
        new_demand = demand_set(bundles, prices, utility, new_budget)
        if (np.array_equal(new_demand, current_demand)):
            pass
        else:
            discontinuities.append(new_budget)
            allocations.append(current_demand)
            current_demand = new_demand
        current_budget += delta

    discontinuities.append(budget+epsilon)
    allocations.append(demand_set(bundles, prices, utility, budget+epsilon))

    
    #Get intervals from discontinuity points
    intervals = []
    for i in range(len(discontinuities) - 1):
        intervals.append([discontinuities[i], discontinuities[i+1]])
    
    #return intervals, allocations
    return allocations

In [225]:
def clearing_error_optimizer(a, c, p):

    # Define n, m
    # n = no. of studnets
    # m = no. of classes
    n = len(a)
    m = len(c)

    # Create a new model
    model = gp.Model("Clearing_Error_Minimization")

    # Variables

    # clearing error variables
    z = model.addVars(m, vtype=GRB.INTEGER, name="z")
    y = model.addVar(name="y")

    # Indicator variable for budget intervals. Modified because k_i might be different for each student
    x = [None]*n
    for i in range(n):
        k = len(a[i])
        x[i] = model.addVars(k, vtype=GRB.BINARY)


    # OBJECTIVE: minimize y, which will be equal to the 1-norm of z
    model.setObjective(y, sense=GRB.MINIMIZE)


    # Constraints

    #Set y == ||z||_1
    model.addGenConstrNorm(y, z, 1.0, "normconstr")

    # One schedule per student
    for i in range(n):
        k = len(x[i])
        model.addConstr(gp.quicksum(x[i][l] for l in range(k)) == 1)

    # Integral allocations already accounted for because x variables are GRB.BINARY

    # Clearing error constraints:
    # gp.quicksum(x[i][l] * a[i][l][j] for i in range(n) for l in range(len(x[i])))
    # Note the extra 0 because the demand set is returned as a list
    for j in range(m):
        if (p[j] > 0):
            model.addConstr(gp.quicksum(x[i][l]*a[i][l][0][j] for i in range(n) for l in range(len(x[i]))) == c[j] + z[j])
        elif (p[j] == 0):
            model.addConstr(gp.quicksum(x[i][l]*a[i][l][0][j] for i in range(n) for l in range(len(x[i])) ) <= c[j] + z[j])
        
    model.optimize()

## A-CEEI

Inputs 
* Student's utility functions $u_i$
* Course capacities $c$
* Initial budgets b_0

Parameters
* Step size $\delta$
* Max. budget perturbation $\epsilon$

Outputs
* Final prices $p^* \in \mathbb{R}^m$
* Final budgets $b^* \in \mathbb{R}^n$

### Description

1) Take in $u, c, b_0$.
2) Set p = 0.
3) Find optimal budget perturbation given current prices.
4) With this set of budgets, compute clearing error.
5) Terminate if clearing error is 0. Otherwise, update prices.

In [222]:
bundles = allocations(max_length=2)
a = []
a.append(budget_intervals(bundles, [6, 5, 4, 1], u[0], 10, 0.5, 1))
print(a[0][0])

[array([1., 0., 0., 1.])]
[array([1., 0., 0., 1.])]
[array([1., 0., 1., 0.])]
[array([1., 0., 1., 0.])]
[array([1., 1., 0., 0.])]
[array([1., 1., 0., 0.])]
[[1. 0. 0. 1.]]


In [ ]:
def ACEEI(u, c, b_0, K, delta, epsilon):
    p = np.zeros(len(c))
    bundles = allocations(max_length=K)
    
    zeroClearingError = False

    while(not zeroClearingError):
        a = []
        for i in range(len(u)):
            demands = budget_intervals(bundles, p, u[i], b_0[i], delta, epsilon)
            a.append(demands)

        x, z = clearing_error_optimizer(a, c, p)

        error = list(z.values())

        zeroClearingError = True
        for i in error:
            if (i != 0):
                zeroClearingError = False
        
        if (zeroClearingError):
            print("\n\n\n\nExact equilibrium found")
            return p, x, z
        else:
            p = np.add(p, delta*np.array(list(z.values())))

    return p, x, z